### Задание
1. Снять видео на камеру (любую, можно телефона). Выполнить кручение видео с скоростью 10 градусов в секунду относительно центра (в любую сторону). 
2. Видео после выполнения п.1 и вернуть его в исходное состояние. 
3. Сконвертируйте видео в mp4 при помощи ffmpeg

### Часть 1

Запишем 36 секундный ролик для того, чтобы увидеть полное вращение камеры. Для этого нам потребуется отсчитать количество кадров равное fps * 36.

In [11]:
import cv2

cap = cv2.VideoCapture(1)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
center = (frame_width // 2, frame_height // 2)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('./assets/rotation_output.mp4', fourcc, fps, (frame_width, frame_height))
frame_index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if (not ret) or (frame_index > fps * 36):
        break

    angle = (frame_index / fps) * 10
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_frame = cv2.warpAffine(frame, rotation_matrix, (frame_width, frame_height))
    out.write(rotated_frame)
    frame_index += 1
cap.release()
out.release()

Так как видео уже в mp4 формате, то конвертируем его в gif формат и попробуем добавить для отображения на git

In [ ]:
! ffmpeg -i ./assets/rotation_output.mp4 ./assets/rotation_output.gif

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

![Попытаемся вставить видео как картинку](./assets/rotation_output.gif)

### Часть 2
Зная скорость поворота "стабилизируем" видео вращая его с той же скоростью в обратную сторону.

In [13]:
import cv2

cap = cv2.VideoCapture('./assets/rotation_output.mp4')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
center = (frame_width // 2, frame_height // 2)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('./assets/stabilized_output.mp4', fourcc, fps, (frame_width, frame_height))
frame_index = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    angle = - (frame_index / fps) * 10
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_frame = cv2.warpAffine(frame, rotation_matrix, (frame_width, frame_height))
    out.write(rotated_frame)
    frame_index += 1
cap.release()
out.release()

Так же сконвертируем "стабилизированное" видео

In [14]:
! ffmpeg -i ./assets/stabilized_output.mp4 ./assets/stabilized_output.gif

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

![Попытаемся вставить видео как картинку](./assets/stabilized_output.gif)

### Выводы
Изучили базовые возможности библиотеки OpenCV